## Frequent Aspect Extraction in ReLi
Baseline is a adaptation for the baseline used in he opinion target extraction (OTE) for 2014, 2015 and 2016 versions of SemEval Aspect-Based Sentiment Analysis Task.

Source:

Pontiki, M., Galanis, D., Pavlopoulos, J., Papageorgiou, H., Androutsopoulos, I., & Manandhar, S. (2014, August). Semeval-2014 task 4: Aspect based sentiment analysis. In Proceedings of the 8th international workshop on semantic evaluation (SemEval 2014) (pp. 27-35).

Pontiki, M., Galanis, D., Papageorgiou, H., Manandhar, S., & Androutsopoulos, I. (2015, June). Semeval-2015 task 12: Aspect based sentiment analysis. In Proceedings of the 9th International Workshop on Semantic Evaluation (SemEval 2015), Association for Computational Linguistics, Denver, Colorado (pp. 486-495).

In [2]:
from __future__ import unicode_literals
from __future__ import division
from __future__ import print_function

In [1]:
from lxml import etree
parser = etree.XMLParser(remove_blank_text=True)
trainset = etree.parse('../corpus/ReLi_train.xml', parser)
testset = etree.parse('../corpus/ReLi_test.xml', parser)

## 1. All aspects in trainset

In [3]:
from collections import Counter
targets = Counter([opinion_node.get('target').lower()
                    for opinion_node in trainset.iter('Opinion')
                    if opinion_node.get('target') != 'NULL'
                   ])

In [4]:
import ipy_table

data = [['freq', '%freq', 'target']]
for target, freq in targets.most_common(20):
    ratio = freq / sum(targets.values()) *100
    data.append([freq, '{:.1f}%'.format(ratio), target])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

freq,%freq,target
681,32.9%,livro
150,7.2%,história
88,4.2%,leitura
59,2.8%,personagens
47,2.3%,crepúsculo
44,2.1%,narrativa
42,2.0%,romance
39,1.9%,obra
35,1.7%,1984
35,1.7%,final


In [5]:
# Build a regex to match the targets in the text
import re
targets_list = sorted(list(targets))
targets_list.sort(key=len, reverse=True)
targets_pattern = r'\b(' + '|'.join([re.escape(t) for t in targets_list]) + r')\b'
len(targets_list)

436

In [6]:
test_gold = list()
prediction = list()

for sentence_node in testset.iter('sentence'):    
    sentence_opinions = []
    for opinion_node in sentence_node.iter('Opinion'):
        target = opinion_node.get('target')
        start = int(opinion_node.get('from'))
        end = int(opinion_node.get('to'))
        # evaluation explicit says to discart NULL values
        if target != 'NULL':
            sentence_opinions.append((target, start, end))
    test_gold.append(sentence_opinions)
    
    text = sentence_node.xpath('./text/text()')[0]
    text_opinions = []
    
    for m in re.finditer(targets_pattern, text, flags=re.I):
        text_opinions.append( (m.group(), m.start(), m.end()) )
    prediction.append(text_opinions)

In [8]:
data = [['Gold Standard', 'Predicted', 'Sentence']]
for index, (gold, pred) in enumerate(list(zip(test_gold, prediction))[:20]):
    sentence = list(testset.iter('sentence'))[index].xpath('./text/text()')[0]
    data.append([gold, pred, sentence])

ipy_table.make_table(data)
ipy_table.set_global_style(wrap=True)
ipy_table.apply_theme('basic')

Gold Standard,Predicted,Sentence
[],"[('livro', 30, 35), ('história', 48, 56)]",Está provado: Pode existir um livro bom sem uma história boa
[],"[('a', 40, 41), ('a', 107, 108), ('a', 121, 122)]","""Se querem mesmo ouvir o que aconteceu, a primeira coisa que vão querer saber é onde eu nasci, como passei a porcaria de a minha infância (...) e toda essa lenga-lenga...""."
"[('O Apanhador em o Campo de Centeio', 14, 47)]","[('O Apanhador em o Campo de Centeio', 14, 47), ('a', 76, 77), ('literatura', 78, 88)]","Assim começa ""O Apanhador em o Campo de Centeio"", um verdadeiro clássico de a literatura mundial."
[],"[('a', 116, 117), ('a', 148, 149), ('a', 183, 184), ('AUTOR', 198, 203)]","E essas três primeiras linhas são suficientes para termos um norte bem contextualizado de tudo quanto encontraremos a o longo de as 207 páginas que a versão brasileira, publicada por a EDITORA DE O AUTOR, trará."
[],"[('Apanhador', 6, 15), ('a', 39, 40), ('a', 48, 49), ('Holden Caulfield', 84, 100), ('anos', 128, 132), ('esse', 141, 145)]","Em ""O Apanhador"", o leitor é convidado a vestir a pele de um adolescente revoltado, Holden Caulfield, que, em o auge de seus 17 anos, conta ""esse negócio doido que aconteceu em o último Natal""."
[],"[('A', 0, 1)]",A verdade é que não tem negócio doido coisa nenhuma.
[],"[('A', 0, 1), ('história', 2, 10), ('a', 55, 56), ('história', 57, 65), ('literatura', 160, 170), ('este', 179, 183), ('a', 205, 206), ('autor', 246, 251), ('a', 307, 308), ('a', 353, 354)]","A história em si não tem nada de mirabolante, se é que a história despretensiosa de um adolescente rebelde que bombou em quase todas as matérias (exceto Inglês/literatura, e é em este ponto que identifico a primeira marquinha de o alter ego de o autor, J. D. Salinger), que tinha um olhar crítico para tudo a o seu redor, e ainda parecia bem perdido em a vida, soaria mirabolante ou diferente de o cotidiano de boa parte de os adolescentes de os dias de hoje."
[],"[('a', 33, 34)]","Mas vamos devagar com o andar de a carruagem, chegaremos lá."
[],"[('este', 23, 27), ('ser', 56, 59)]",Há de se ressaltar que este clássico não é tão fácil de ser encontrado.
[],"[('ser', 72, 75)]","Antes de achar o meu exemplar, procurei em várias livrarias, que diziam ser necessário fazer uma encomenda, com todo aquele papo de ""tantos dias úteis"", ""ligamos para você quando chegar"", etc e tal."


### Aspect-extraction Evaluation methodology

Pontiki, M., Galanis, D., Papageorgiou, H., Manandhar, S., & Androutsopoulos, I. (2015, June). Semeval-2015 task 12: Aspect based sentiment analysis. In Proceedings of the 9th International Workshop on Semantic Evaluation (SemEval 2015), Association for Computational Linguistics, Denver, Colorado (pp. 486-495).

http://www.anthology.aclweb.org/S/S15/S15-2082.pdf

From 4.1 Evaluation Measures, page 491:

Slot 2: F-1 scores are calculated by comparing
the targets that a system returned (for all the sentences)
to the corresponding gold targets (using
micro-averaging). The targets are extracted using
their starting and ending offsets. The calculation
for each sentence considers only distinct targets
and discards NULL targets, since they do not correspond
to explicit mentions

In [9]:
# Micro-averaged Precision
correct = 0
total = 0
for index in range(len(list(testset.iter('sentence')))):
    correct += len([x for x in test_gold[index] if x in prediction[index]])
    total += len(prediction[index])

precision = 100 * correct / total
print('Precision: {:.2f}%'.format(precision))

Precision: 7.14%


In [10]:
# Micro-averaged Recall
correct = 0
total = 0
for index in range(len(list(testset.iter('sentence')))):
    correct += len([x for x in test_gold[index] if x in prediction[index]])
    total += len(test_gold[index])

recall = 100* correct / total
print('Recall: {:.2f}%'.format(recall))

Recall: 82.26%


In [11]:
print('F-measure: {:.2f}%'.format((2 * precision * recall) / (precision + recall)))

F-measure: 13.13%


In [13]:
# Save the prediction (Optional)
import re
for sentence_node in testset.iter('sentence'):
    sentence_opinions = []
    opinions_node = sentence_node.xpath('./Opinions')
    if opinions_node:
        opinions_node = opinions_node[0]
    else:
        opinions_node = etree.SubElement(sentence_node, 'Opinions')
        
    for opinion_node in sentence_node.xpath('./Opinions/Opinion'):
        opinions_node.remove(opinion_node)
    
    text = sentence_node.xpath('./text/text()')[0]
    for m in re.finditer(targets_pattern, text):
        opinion_node = etree.SubElement(opinions_node, 'Opinion')
        opinion_node.set('target', m.group())
        opinion_node.set('from', str(m.start()))
        opinion_node.set('to', str(m.end()))
        
etree.ElementTree(testset.getroot()).write('../corpus/pred.xml',encoding='utf8', xml_declaration=True, pretty_print=True)

## 2. All aspects in trainset removing stopwords

In [12]:
from collections import Counter
from nltk.corpus import stopwords
# build the targets
stopwords = stopwords.words('portuguese') + ['esse', 'ser', 'ele', 'isso']
targets = Counter([opinion_node.get('target').lower()
                    for opinion_node in trainset.iter('Opinion')
                    if opinion_node.get('target') != 'NULL' and
                       opinion_node.get('target').lower() not in stopwords
                   ])

# print the targets
import ipy_table
data = [['freq', '%freq', 'target']]
for target, freq in targets.most_common(20):
    ratio = freq / sum(targets.values()) *100
    data.append([freq, '{:.1f}%'.format(ratio), target])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

freq,%freq,target
681,33.9%,livro
150,7.5%,história
88,4.4%,leitura
59,2.9%,personagens
47,2.3%,crepúsculo
44,2.2%,narrativa
42,2.1%,romance
39,1.9%,obra
35,1.7%,1984
35,1.7%,final


In [13]:
import re

# function to evaluate targets in testset and return precision, recall and f-measure
def evaluate(targets):

    # Build a regex to match the targets in the text
    targets_list = sorted(list(targets))
    targets_list.sort(key=len, reverse=True)
    targets_pattern = r'\b(' + '|'.join([re.escape(t) for t in targets_list]) + r')\b'

    test_gold = list()
    prediction = list()

    for sentence_node in testset.iter('sentence'):    
        sentence_opinions = []
        for opinion_node in sentence_node.iter('Opinion'):
            target = opinion_node.get('target')
            start = int(opinion_node.get('from'))
            end = int(opinion_node.get('to'))
            # evaluation explicit says to discart NULL values
            if target != 'NULL':
                sentence_opinions.append((target, start, end))
        test_gold.append(sentence_opinions)

        text = sentence_node.xpath('./text/text()')[0]
        text_opinions = []

        for m in re.finditer(targets_pattern, text, flags=re.I):
            text_opinions.append( (m.group(), m.start(), m.end()) )
        prediction.append(text_opinions)
        
    # Micro-averaged Precision
    correct = 0
    total = 0
    for index in range(len(list(testset.iter('sentence')))):
        correct += len([x for x in test_gold[index] if x in prediction[index]])
        total += len(prediction[index])

    precision = 100 * correct / total    

    # Micro-averaged Recall
    correct = 0
    total = 0
    for index in range(len(list(testset.iter('sentence')))):
        correct += len([x for x in test_gold[index] if x in prediction[index]])
        total += len(test_gold[index])

    recall = 100* correct / total
    
    # F-measure
    if precision + recall != 0:
        fmeasure = (2 * precision * recall) / (precision + recall)
    else:
        fmeasure = 0
    
    return (precision, recall, fmeasure)
    
  

In [14]:
precision, recall, fmeasure = evaluate(targets)
print('Precision: {:.2f}%'.format(precision))
print('Recall: {:.2f}%'.format(recall))
print('F-measure: {:.2f}%'.format(fmeasure))

Precision: 14.32%
Recall: 79.11%
F-measure: 24.25%


## 3. All aspects in trainset with a cut in frequency

In [15]:
# build the targets
targets = Counter([opinion_node.get('target').lower()
                    for opinion_node in trainset.iter('Opinion')
                    if opinion_node.get('target') != 'NULL'])

data = [['cut', 'number of targets', 'precision', 'recall', 'f-measure']]
for min_freq in range(0,11,1):
    target_list = [target for target, freq in targets.items() if freq/sum(targets.values()) >= min_freq/100]
    precision, recall, fmeasure = evaluate(target_list)
    data.append(['{:.1f}%'.format(min_freq), 
                 len(target_list),
                 '{:.2f}%'.format(precision), 
                 '{:.2f}%'.format(recall), 
                 '{:.2f}%'.format(fmeasure)])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

cut,number of targets,precision,recall,f-measure
0.0%,436,7.14%,82.26%,13.13%
1.0%,15,25.66%,64.95%,36.79%
2.0%,7,30.27%,55.51%,39.17%
3.0%,3,31.39%,45.35%,37.10%
4.0%,3,31.39%,45.35%,37.10%
5.0%,2,31.85%,41.92%,36.20%
6.0%,2,31.85%,41.92%,36.20%
7.0%,2,31.85%,41.92%,36.20%
8.0%,1,33.57%,33.62%,33.60%
9.0%,1,33.57%,33.62%,33.60%


## 4. All aspects with relative frequency
 
The token must happen % of the time annotated as aspect in the text.

In [16]:
sentences = ' '.join([node.get('form').lower() for node in testset.iter('word')])
freqlist =  Counter([node.get('target').lower() for node in testset.iter('Opinion')])

# build the targets
targets = Counter([opinion_node.get('target').lower()
                    for opinion_node in trainset.iter('Opinion')
                    if opinion_node.get('target') != 'NULL'])

data = [['cut', 'number of targets', 'precision', 'recall', 'f-measure']]
for min_freq in range(0,100,5):
    
    target_list = [target for target, freq in targets.items() if freq/max(sentences.count(' ' + target + ' '),0.00001) >= min_freq/100]
        
    precision, recall, fmeasure = evaluate(target_list)
    data.append(['{:.1f}%'.format(min_freq), 
                 len(target_list),
                 '{:.2f}%'.format(precision), 
                 '{:.2f}%'.format(recall), 
                 '{:.2f}%'.format(fmeasure)])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

cut,number of targets,precision,recall,f-measure
0.0%,436,7.14%,82.26%,13.13%
5.0%,406,16.65%,80.11%,27.57%
10.0%,390,19.03%,78.25%,30.61%
15.0%,373,21.94%,74.82%,33.93%
20.0%,367,22.74%,74.11%,34.80%
25.0%,344,25.34%,70.24%,37.24%
30.0%,340,25.71%,70.24%,37.64%
35.0%,320,26.25%,68.53%,37.96%
40.0%,318,26.57%,67.95%,38.20%
45.0%,318,26.57%,67.95%,38.20%


## 5. Retrieve Aspects with syntatic and semantic restriction

In [19]:
from lxml import etree
parser = etree.XMLParser(remove_blank_text=True)
trainset = etree.parse('../corpus/ReLiPalavras_train.xml', parser)
testset = etree.parse('../corpus/ReLiPalavras_test.xml', parser)

In [20]:
def restriction(word_node):
    return True

In [53]:
from sklearn.metrics import precision_recall_fscore_support
# function to evaluate targets in testset and return precision, recall and f-measure
def evaluate(targets, restriction):
    
    test_gold = list()
    prediction = list()
    golds, guesses = [], []

    for word_node in testset.iter('word'):
        
        if word_node.get('opinion','O') == 'O':
            golds.append(False)
        else:
            golds.append(True)
            
        if word_node.get('form').lower() in targets and restriction(word_node):
            guesses.append(True)
        else:
            guesses.append(False)                

    
    return precision_recall_fscore_support(golds, guesses)

In [36]:
def restriction(word_node):
    if ((word_node.get('postag') == 'n' or word_node.get('postag') == 'prop') and
        tag != 'PU' and 
        tag != 'DN' and 
        tag != 'CO' and 
        tag != 'fA' and 
        tag != 'Op'):
        return True
    return False

freqlist =  Counter([node.get('form').lower() for node in trainset.iter('word')])

from operator import itemgetter
srl_freq = dict()
freqlist = dict()

for word_node in trainset.iter('word'):
    form = word_node.get('form').lower()
    tag = word_node.get('deprel')
    if word_node.get('obj') != 'O' and restriction(word_node):
        srl_freq[tag] = srl_freq.get(tag, 0) + 1
        if tag not in freqlist:
            freqlist[tag] = dict()
        freqlist[tag][form] = freqlist[tag].get(form, 0) + 1
    
data = [['FREQ', '% FREQ', 'DEPREL TAG', 'EXAMPLES']]

targets = {}
for tag, freq in sorted(srl_freq.items(), key=itemgetter(1), reverse=True):
    examples = ', '.join(['{}({:.1f}%)'.format(w,f / sum(freqlist[tag].values()) * 100) 
                          for w,f in sorted(freqlist[tag].items(), key=itemgetter(1), reverse=True)[:5]])
    targets.update(freqlist[tag])
    ratio = freq / sum(srl_freq.values()) * 100
    data.append([freq, '{:.2f}%'.format(ratio), tag, examples])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

FREQ,% FREQ,DEPREL TAG,EXAMPLES
679,33.75%,S,"livro(31.8%), história(9.1%), leitura(4.1%), crepúsculo(3.8%), personagens(3.5%)"
527,26.19%,DP,"livro(17.8%), história(6.1%), personagens(4.4%), personagem(2.8%), final(2.7%)"
227,11.28%,CJT,"livro(23.8%), leitura(7.5%), história(7.5%), personagens(4.4%), narrativa(4.0%)"
219,10.88%,Od,"livro(46.1%), história(7.8%), leitura(5.9%), obra(2.3%), final(1.8%)"
184,9.15%,Cs,"livro(60.3%), história(8.2%), obra(5.4%), leitura(4.9%), romance(4.3%)"
113,5.62%,X,"livro(60.2%), história(8.8%), leitura(7.1%), romance(3.5%), obra(2.7%)"
30,1.49%,fCs,"livro(33.3%), história(20.0%), personagens(16.7%), romance(6.7%), narrativa(6.7%)"
10,0.50%,DNapp,"dora(10.0%), planos(10.0%), vida(10.0%), confusões(10.0%), reviravoltas(10.0%)"
8,0.40%,fAtop,"livro(75.0%), narração(12.5%), grande_irmão(12.5%)"
8,0.40%,DNc,"livro(37.5%), professor(25.0%), enredo(12.5%), trama(12.5%), início(12.5%)"


In [37]:
# print the targets
data = [['freq', '%freq', 'target']]
for target, freq in sorted(targets.items(), key=itemgetter(1), reverse=True):
    ratio = freq / sum(targets.values()) *100
    data.append([freq, '{:.1f}%'.format(ratio), target])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

freq,%freq,target
12,2.6%,cegueira
9,2.0%,areia
6,1.3%,história
5,1.1%,livros
5,1.1%,parte
5,1.1%,meninos
4,0.9%,crepúsculo
4,0.9%,fim
4,0.9%,autora
4,0.9%,final


In [59]:
data = [['cut', 'number of targets', 'precision', 'recall', 'f-measure']]
for min_freq in range(0,5,1):
    target_list = [target for target, freq in targets.items() if freq >= min_freq]
    precision, recall, fmeasure, support = evaluate(target_list, restriction)    
    data.append(['{}'.format(min_freq), 
                 len(target_list),
                 '{:.2f}%'.format(precision[1]*100), 
                 '{:.2f}%'.format(recall[1]*100), 
                 '{:.2f}%'.format(fmeasure[1]*100)])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

cut,number of targets,precision,recall,f-measure
0,348,6.05%,6.83%,6.41%
1,348,6.05%,6.83%,6.41%
2,61,6.73%,2.38%,3.51%
3,20,8.42%,1.55%,2.62%
4,11,9.29%,1.40%,2.43%


In [65]:
from operator import itemgetter
srl_freq = dict()
freqlist = dict()

for word_node in trainset.iter('word'):
    form = word_node.get('form').lower()
    tag = word_node.get('srl')
    if word_node.get('obj') != 'O' and tag and (word_node.get('postag') == 'n' or word_node.get('postag') == 'prop'):
        srl_freq[tag] = srl_freq.get(tag, 0) + 1
        if tag not in freqlist:
            freqlist[tag] = dict()
        freqlist[tag][form] = freqlist[tag].get(form, 0) + 1 
    
data = [['FREQ', '% FREQ', 'SRL TAG', 'EXAMPLES']]

for tag, freq in sorted(srl_freq.items(), key=itemgetter(1), reverse=True):
    examples = ', '.join(['{}({:.1f}%)'.format(w,f / sum(freqlist[tag].values()) * 100) 
                          for w,f in sorted(freqlist[tag].items(), key=itemgetter(1), reverse=True)[:5]])
    ratio = freq / sum(srl_freq.values()) * 100
    data.append([freq, '{:.2f}%'.format(ratio), tag, examples])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

FREQ,% FREQ,SRL TAG,EXAMPLES
664,40.19%,TH,"livro(35.7%), história(9.2%), crepúsculo(3.9%), leitura(3.6%), personagens(3.2%)"
221,13.38%,ATR,"livro(54.3%), história(9.0%), romance(4.5%), obra(4.5%), leitura(4.5%)"
164,9.93%,PAT,"livro(34.1%), história(6.7%), personagens(6.1%), leitura(4.3%), obra(2.4%)"
164,9.93%,AG,"livro(28.0%), história(7.3%), leitura(6.1%), edward(3.0%), crepúsculo(3.0%)"
113,6.84%,DENOM,"livro(60.2%), história(8.8%), leitura(7.1%), romance(3.5%), obra(2.7%)"
68,4.12%,LOC,"livro(22.1%), cegueira(17.6%), páginas(4.4%), campo(4.4%), parte(2.9%)"
50,3.03%,CAU,"livro(34.0%), história(14.0%), modo(4.0%), allie(2.0%), phoebe(2.0%)"
39,2.36%,LOC-TMP,"começo(23.1%), história(20.5%), final(20.5%), crepúsculo(17.9%), início(7.7%)"
27,1.63%,COM-ADV,"história(18.5%), enredo(14.8%), histórias(7.4%), narrativa(7.4%), estória(7.4%)"
25,1.51%,ID,"bella(16.0%), personagens(8.0%), antolini(4.0%), gato(4.0%), planos(4.0%)"


In [80]:
freqlist =  Counter([node.get('form').lower() for node in trainset.iter('word')])

from operator import itemgetter
srl_freq = dict()
freqlist = dict()

for word_node in trainset.iter('word'):
    form = word_node.get('form').lower()
    tag = word_node.get('srl')
    if word_node.get('obj') != 'O' and tag  and (word_node.get('postag') == 'n' or word_node.get('postag') == 'prop'):
        srl_freq[tag] = srl_freq.get(tag, 0) + 1
        if tag not in freqlist:
            freqlist[tag] = dict()
        freqlist[tag][form] = freqlist[tag].get(form, 0) + 1 

targets = {}
for tag, freq in sorted(srl_freq.items(), key=itemgetter(1), reverse=True):
    if freq >= 7:        
        targets.update([(w,f) for w,f in freqlist[tag].items() if (f / sum(freqlist[tag].values()) * 100) >= 0.30])
        
# print the targets
data = [['freq', '%freq', 'target']]
for target, freq in sorted(targets.items(), key=itemgetter(1), reverse=True):
    ratio = freq / sum(targets.values()) *100
    data.append([freq, '{:.1f}%'.format(ratio), target])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

freq,%freq,target
12,4.3%,cegueira
4,1.4%,bella
4,1.4%,autora
3,1.1%,esse
3,1.1%,páginas
3,1.1%,início
3,1.1%,holden
3,1.1%,se_houver_amanhã
3,1.1%,o_reverso_de_a_medalha
3,1.1%,vocabulário


In [79]:
data = [['cut', 'number of targets', 'precision', 'recall', 'f-measure']]
for min_freq in range(12):
    target_list = [target for target, freq in targets.items() if freq >= min_freq]
    precision, recall, fmeasure = evaluate(target_list)
    data.append(['{:.1f}%'.format(min_freq), 
                 len(target_list),
                 '{:.2f}%'.format(precision), 
                 '{:.2f}%'.format(recall), 
                 '{:.2f}%'.format(fmeasure)])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

cut,number of targets,precision,recall,f-measure
0.0%,290,9.87%,75.38%,17.46%
1.0%,290,9.87%,75.38%,17.46%
2.0%,34,10.54%,16.00%,12.71%
3.0%,11,3.23%,2.00%,2.47%
4.0%,3,3.47%,0.77%,1.26%
5.0%,1,0.00%,0.00%,0.00%
6.0%,1,0.00%,0.00%,0.00%
7.0%,1,0.00%,0.00%,0.00%
8.0%,1,0.00%,0.00%,0.00%
9.0%,1,0.00%,0.00%,0.00%


In [67]:
def restriction(word_node):
    if word_node.get('sem') in sem_list:
        return True
    return False

freqlist =  Counter([node.get('form').lower() for node in trainset.iter('word')])

from operator import itemgetter
srl_freq = dict()
freqlist = dict()

for word_node in trainset.iter('word'):
    form = word_node.get('form').lower()
    tag = word_node.get('sem')
    if word_node.get('obj') != 'O' and tag:
        srl_freq[tag] = srl_freq.get(tag, 0) + 1
        if tag not in freqlist:
            freqlist[tag] = dict()
        freqlist[tag][form] = freqlist[tag].get(form, 0) + 1
    
data = [['FREQ', '% FREQ', 'SEM TAG', 'EXAMPLES']]

sem_list = list()
targets = {}
for tag, freq in sorted(srl_freq.items(), key=itemgetter(1), reverse=True):
    examples = ', '.join(['{}({:.1f}%)'.format(w,f / sum(freqlist[tag].values()) * 100) 
                          for w,f in sorted(freqlist[tag].items(), key=itemgetter(1), reverse=True)[:5]])
    targets.update(freqlist[tag])
    sem_list.append(tag)
    ratio = freq / sum(srl_freq.values()) * 100
    data.append([freq, '{:.2f}%'.format(ratio), tag, examples])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

FREQ,% FREQ,SEM TAG,EXAMPLES
664,41.76%,sem-r,"livro(90.7%), romance(5.9%), estória(1.2%), livros(0.8%), crônicas(0.3%)"
152,9.56%,per domain sem-r,"história(95.4%), histórias(4.6%)"
88,5.53%,Hmyth Hattr,"personagens(70.5%), personagem(28.4%), personagem-adolescente(1.1%)"
64,4.03%,sem-r activity,leitura(100.0%)
37,2.33%,sem-c Lh activity,"obra(97.3%), obras(2.7%)"
35,2.20%,sem-c,"enredo(71.4%), ficção(8.6%), idéia(5.7%), ideia(5.7%), idéias(2.9%)"
33,2.08%,sem-s sem-r,narrativa(100.0%)
31,1.95%,event temp,final(100.0%)
23,1.45%,temp,"começo(69.6%), início(30.4%)"
22,1.38%,sem-c cord act-s,"trama(90.9%), tramas(9.1%)"


In [74]:
# print the targets
data = [['freq', '%freq', 'target']]
for target, freq in sorted(targets.items(), key=itemgetter(1), reverse=True):
    ratio = freq / sum(targets.values()) *100
    data.append([freq, '{:.1f}%'.format(ratio), target])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

freq,%freq,target
602,38.4%,livro
145,9.2%,história
64,4.1%,leitura
62,4.0%,personagens
39,2.5%,romance
36,2.3%,obra
33,2.1%,narrativa
31,2.0%,final
25,1.6%,personagem
25,1.6%,enredo


In [73]:
data = [['cut', 'number of targets', 'precision', 'recall', 'f-measure']]
for min_freq in range(0,5,1):
    target_list = [target for target, freq in targets.items() if freq >= min_freq]
    precision, recall, fmeasure, support = evaluate(target_list, restriction)    
    data.append(['{}'.format(min_freq), 
                 len(target_list),
                 '{:.2f}%'.format(precision[1]*100), 
                 '{:.2f}%'.format(recall[1]*100), 
                 '{:.2f}%'.format(fmeasure[1]*100)])

ipy_table.make_table(data)
ipy_table.apply_theme('basic')

cut,number of targets,precision,recall,f-measure
0,259,6.72%,6.00%,6.34%
1,259,6.72%,6.00%,6.34%
2,98,7.47%,4.73%,5.79%
3,52,7.90%,4.15%,5.44%
4,41,8.41%,3.98%,5.40%
